#### Provide the data in following naming convention in 'data' directory:

- 2D observed velocity field FITS file: `<galaxy_name>_1mom.fits`
- Bbarolo fit parameters text file: `<galaxy_name>_ring_parameters.txt`
- Stellar velocity curve: `<galaxy_name>_stellar_velocities.txt`
- Gas velocity_curve: `<galaxy_name>_gas_velocities.txt`

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


from astropy.io import fits
from emcee import EnsembleSampler
import numpy as np
import os
import sys

pwd = os.getcwd()
little_things_root_dir = os.path.dirname(pwd)
sys.path.append(little_things_root_dir)

from little_things_lib.galaxy import Galaxy
from little_things_lib.mcmc_fitter import (
    EmceeParameters,
    get_mcmc_start_position, 
    generate_nwalkers_start_points, 
    lnprob)


## Enter parameters for galaxy in cell below

In [2]:
galaxy_name = 'DDO46'

X_DIM, Y_DIM = 150,150
DEG_PER_PIXEL = 4.17e-4
DISTANCE = 6100 # kpc

# can leave these as any number for now, not used
LUMINOSITY = 1.61e5  # solar luminositiy
HI_MASS = 22.08e7  # solar masses

In [3]:
mcmc_params = EmceeParameters(
    ndim=4,
    nwalkers=10, 
    nburn=50,
    niter=200,
    nthin=0,
    nthreads=4
)

In [4]:
"""
Expect the data to be provided in following naming convention in 'data' directory:

2D observed velocity field FITS file: <galaxy_name>_1mom.fits
Bbarolo fit parameters text file: <galaxy_name>_ring_parameters.txt
Stellar velocity curve: <galaxy_name>_stellar_velocities.txt
Gas velocity_curve: <galaxy_name>_gas_velocities.txt

"""


data_dir = os.path.join(little_things_root_dir, 'data')

observed_2d_vel_field_fits_file = os.path.join(data_dir, f'{galaxy_name}_1mom_crop.fits')
ring_parameters_file = os.path.join(data_dir, 'ringlog2.txt')

stellar_velocities_file = os.path.join(data_dir, f'{galaxy_name}-Stellar-Contribution.txt')
gas_velocities_file = os.path.join(data_dir, f'{galaxy_name}-Gas-Contribution.txt')


In [5]:
radii_kpc, test_rotation_curve, inclinations, position_angles, x_centers, y_centers , v_systemics = \
    np.loadtxt(ring_parameters_file, usecols=(0,2,4,5,-4,-3,-2)).T

avg_inclination = np.mean(inclinations)
avg_position_angle = np.mean(position_angles)
avg_x_center = np.mean(x_centers)
avg_y_center = np.mean(y_centers)
v_systemic = np.mean(v_systemics)

observed_2d_vel_field = fits.open(observed_2d_vel_field_fits_file)[0].data

stellar_radii, stellar_vel = np.loadtxt(stellar_velocities_file, unpack=True, usecols=0), np.loadtxt(stellar_velocities_file, unpack=True, usecols=1)
gas_radii, gas_vel = np.loadtxt(gas_velocities_file, unpack=True, usecols=0), np.loadtxt(gas_velocities_file, unpack=True, usecols=1) 

In [6]:
# all values are just placeholders for testing that functions work
#stellar_radii = np.linspace(0, 450, 41)
#gas_radii = np.linspace(0, 450, 41)
#stellar_vel = np.linspace(0, 450, 41)
#gas_vel = np.linspace(0, 450, 41)

In [7]:
galaxy = Galaxy(
    distance_to_galaxy=DISTANCE,  # [kpc] Look this up for the galaxy 
    deg_per_pixel=DEG_PER_PIXEL ,  # 'CRDELT1' and 'CRDELT2' in the FITS file header (use absolute value)
    image_xdim=X_DIM,
    image_ydim=Y_DIM,
    galaxy_name=galaxy_name,
    vlos_2d_data=observed_2d_vel_field,
    output_dir='output',
    luminosity=LUMINOSITY,
    HI_mass=HI_MASS)

tilted_ring_params = {
    'v_systemic': v_systemic,
    'radii': radii_kpc,   
    'inclination': inclinations,
    'position_angle': position_angles,
    'x_pix_center': x_centers,
    'y_pix_center': y_centers
}

galaxy.set_tilted_ring_parameters(**tilted_ring_params)

galaxy.interpolate_baryonic_rotation_curve(
    baryon_type='stellar',
    rotation_curve_radii=stellar_radii,
    rotation_curve_velocities=stellar_vel)

galaxy.interpolate_baryonic_rotation_curve(
    baryon_type='gas',
    rotation_curve_radii=gas_radii,
    rotation_curve_velocities=gas_vel)

# initialize MCMC start position and bounds
galaxy.set_prior_bounds(
    cross_section_bounds=(1e-5, 1e-4),
    rate_bounds=(2, 1e4),
    sigma0_bounds=(2, 1e3),
    ml_bounds=(0.1, 10),
    ml_median=0.5,
    rmax_prior=False,
    vmax_prior=False,
    log10_rmax_spread=0.11,
    log10_c200_spread = 0.11,
    abs_err_vel_factor=0.05,
    tophat_width=3)

ValueError: A value in x_new is below the interpolation range.

In [ ]:

start_point, start_point_radii = get_mcmc_start_position(galaxy)
start_pos = generate_nwalkers_start_points(mcmc_params.nwalkers, start_point, start_point_radii)

sampler = EnsembleSampler(
    mcmc_params.nwalkers,
    mcmc_params.ndim, 
    lnprob, 
    args=[galaxy], 
    threads=mcmc_params.nthreads)
sampler.run_mcmc(start_pos, mcmc_params.niter)

